In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..')

In [16]:
from src.wess_zumino_model import WessZuminoModel
from src.matrix_to_ps import matrix_to_pse
from src.binary_encodings import standard_encode
from src.qiskit_utilities import pauli_dict_to_op, op_to_trotter


from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import RealAmplitudes
from qiskit import BasicAer
from qiskit import qpy
from qiskit import QuantumCircuit, transpile


import scipy.sparse.linalg
import sympy as sp
import numpy as np

from ipywidgets import IntProgress
from IPython.display import display, clear_output 

import json
import math

In [17]:
def potential(self, n):
    # m*q with m=1
    return -self.mass*self.qs[n]

N=2
wz=WessZuminoModel(N,1.0,potential,'dirichlet')

## TODO 

* Different circuit forms ReAmp/NCount/ExcitationPreserving
* Different entanglement types
* Different rep counts (less reps = faster VQE but less expressive)
* Save plot of minimization?

In [18]:
cutoff=16
wz.construct_ham_matrix(cutoff)
e0=scipy.sparse.linalg.eigs(wz.hamMat,k=1,sigma=0.0)[0]
ps=matrix_to_pse(wz.hamMat, standard_encode)
op=pauli_dict_to_op(ps.to_dict())

nq=math.floor(math.log2(wz.hamMat.shape[0]))
if not math.log2(wz.hamMat.shape[0]).is_integer():
    nq+=1

In [22]:
backend = BasicAer.get_backend("statevector_simulator")

#results={}

vqeRuns=50

style = {'description_width': 'initial'}
layout = {'width':'600px'}

#for entanglement in ['linear','reverse_linear','circular','sca']: #full and reverse linear are same, reverse has less gates
for entanglement in ['reverse_linear','circular','sca']: #full and reverse linear are same, reverse has less gates
    results[entanglement]={}
    for reps in [1,2,4,8]:
        ansatz = RealAmplitudes(nq, entanglement=entanglement, reps=reps)
        vqe = VQE(ansatz=ansatz, optimizer=COBYLA(), quantum_instance=backend)
        
        values=[]
        progress = IntProgress(min=0, max=vqeRuns, description="Running VQE with {} entanglement and {} reps".format(entanglement,reps), style=style, layout=layout)
        display(progress)
        for run in range(vqeRuns):
            values.append(vqe.compute_minimum_eigenvalue(op).eigenvalue.real)
            progress.value += 1
            
        results[entanglement][reps]=values
    
        with open('Data/vqe_linear_real_amplitudes.json','w') as file:
            json.dump(results, file)
        
        clear_output()


In [27]:
results_real_amplitudes=results

In [28]:
from qiskit.circuit.library import ExcitationPreserving

results={}

vqeRuns=50

style = {'description_width': 'initial'}
layout = {'width':'600px'}

for entanglement in ['linear','reverse_linear','circular','sca']: #full and reverse linear are same, reverse has less gates
    results[entanglement]={}
    for reps in [1,2,4,8]:
        ansatz = ExcitationPreserving(nq, entanglement=entanglement, reps=reps)
        vqe = VQE(ansatz=ansatz, optimizer=COBYLA(), quantum_instance=backend)
        
        values=[]
        progress = IntProgress(min=0, max=vqeRuns, description="Running VQE with {} entanglement and {} reps".format(entanglement,reps), style=style, layout=layout)
        display(progress)
        for run in range(vqeRuns):
            values.append(vqe.compute_minimum_eigenvalue(op).eigenvalue.real)
            progress.value += 1
            
        results[entanglement][reps]=values
    
        with open('Data/vqe_linear_excitation_preserving.json','w') as file:
            json.dump(results, file)
        
        clear_output()